In [ ]:
import importlib
import numpy as np
import matplotlib.pyplot as plt

import colors, plot_func, helpers, file_helpers, experiments

## Supervisor

In [ ]:
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Sprvsr = SupervisorClass()
# Sprvsr.init_dataset()

In [ ]:
importlib.reload(file_helpers)

import ForsentekClass
importlib.reload(ForsentekClass)
from ForsentekClass import ForsentekClass

Snsr = ForsentekClass()

## Check force

In [ ]:
t, data = Snsr.measure(2)
plt.plot(t,data)
plt.xlabel(r"$t\,[\mathrm{s}]$")
plt.ylabel(r"$F\,[\mathrm{N}]$")
plt.legend(['x', 'y', 'z'])

In [ ]:
import MecaClass
importlib.reload(MecaClass)
from MecaClass import MecaClass

m = MecaClass()

In [ ]:
m.connect()

In [ ]:
m.move_to_origin()

## Check Robot movement

In [ ]:
# m.robot.MoveLin(140, 0, 180, 180, 0, 0)
points = (140.0, 0.0, 22.5, 179.9, 0.1, 0.1)
m.move_lin(points)

## Training

In [ ]:
# Sprvsr.pos_in_t = np.array([[180, 40, 0], [180, 40, -40], [180, 40, -80], 
#                             [180, 40, -120], [180, 40, -140], [180, 40, -160], 
#                             [180, 40, -180], [180, 40, -200] ])
Sprvsr.pos_in_t = np.array([[180, 70, -90], [180, 75, -90], [180, 80, -90], 
                            [180, 85, -90], [180, 90, -90], [180, 95, -90], 
                            [180, 100, -90], [180, 105, -90]])
Sprvsr.T = np.shape(Sprvsr.pos_in_t)[0]
Sprvsr.F_in_t = np.zeros([Sprvsr.T, 3])

for t in range(Sprvsr.T):
    Sprvsr.draw_measurement(t)
    print('tip position = ', Sprvsr.pos)
    m.move_lin(Sprvsr.pos)
    t_during_meas, force_in_t = Snsr.measure()
    Snsr.mean_force(force_in_t)
    plt.plot(t_during_meas, force_in_t)
    plt.show()
    Sprvsr.F_in_t[t] = Sprvsr.global_force(Snsr)
    print('mean force = ', Sprvsr.F_in_t[t])

In [ ]:
plt.plot(Sprvsr.F_in_t[:,:]/Sprvsr.F_in_t[0,:], '.')
plt.show()

In [ ]:
m.disconnect()

## Logging - fixed arm

In [ ]:
experiments.sweep_measurement_fixed_origami(x_range, y_range, theta_range, N, robot, force_sensor)

## Calibrate forces

In [ ]:
importlib.reload(experiments)

weights = [1, 2, 5, 10, 20, 50, 200]
voltages_vec, forces_vec, stds_vec, fit_params = experiments.calibrate_forces(Snsr, weights)

## End of day

In [ ]:
# m.robot.MoveLin(40, 0, 230, 180, 0, 0)
m.move_to_sleep_pos()

In [ ]:
m.disconnect()